In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import pickle
from itertools import product
from IPython.display import display

In [2]:
import sys 
sys.path.append('/Users/susierwu/Dropbox/DLCA_DynCO2/pdpLCA_GT/') 
from utils import *
from utils.TRL import * 

before running TRL module, you should have empty vehicle dataset prepared already, with TRL assigned, this module is designed to get a TRL dataset with three TRL data variable assigned (TRL score (0-9) prepared already in TRL_1B, TRL_applicability (0 or 1) and TRL_availability (0 or 1)) after proper vehicle mapping, which will be used when building up a future vehicle model. Finally, you can visualize the TRL table


In [3]:
data = pd.read_excel("../data/to_use_interm/TRL_1B_tech_whTRLadded.xlsx",  index_col=[0,1,2,3,4,5,6]) #  total 6 index level
data.head()

TRL
v_type        size       powertrain v_year SSP    Module Tech           
garbage_truck HDV_class7 ICEV-d     2030   ssp119 V1A    dummy_tech    9
                                                  V1B    LIB_NMC622    9
                                                         LIB_NCA       9
                                                         LIB_LFP       9
                                                         LIB_LTO       9

### powertrain_tech mapping, and assign TRL applicability and avail scores, for ICEV-d, the V1B won't apply 

In [4]:
pw_tech_map_dict =  [{'ICEV-d': ['V1A']}, {'ICEV-d': ['F2_fuel_diesel']}, {'ICEV-d': ['V2_oper']} , 
            {'BEV': ['V1A']}, {'BEV': ['V1B']}, {'BEV': ['F2_fuel_elec']}]

In [5]:
data2 = get_avail_tech_for_V(data, pw_tech_map_dict)

In [6]:
data2

TRL  \
v_type        size       powertrain v_year SSP    Module         Tech              
garbage_truck HDV_class7 ICEV-d     2030   ssp119 V1A            dummy_tech    9   
                                                  V1B            LIB_NMC622    9   
                                                                 LIB_NCA       9   
                                                                 LIB_LFP       9   
                                                                 LIB_LTO       9   
...                                                                          ...   
                         BEV        2050   ssp585 V1B            LSB           5   
                                                  F2_fuel_diesel diesel_mix    9   
                                                  F2_fuel_elec   REC_100PV     7   
                                                                 grid_elec     9   
                                                  V2_oper        dummy_tech    9   

                                                                             tech_appl  \
v_type        size       powertrain v_year SSP    Module         Tech                    
garbage_truck HDV_class7 ICEV-d     2030   ssp119 V1A            dummy_tech          1   
                                                  V1B            LIB_NMC622          0   
                                                                 LIB_NCA             0   
                                                                 LIB_LFP             0   
                                                                 LIB_LTO             0   
...                                                                                ...   
                         BEV        2050   ssp585 V1B            LSB                 1   
                                                  F2_fuel_diesel diesel_mix          0   
                                                  F2_fuel_elec   REC_100PV           1   
                                                                 grid_elec           1   
                                                  V2_oper        dummy_tech          0   

                                                                             tech_avail  
v_type        size       powertrain v_year SSP    Module         Tech                    
garbage_truck HDV_class7 ICEV-d     2030   ssp119 V1A            dummy_tech           1  
                                                  V1B            LIB_NMC622           0  
                                                                 LIB_NCA              0  
                                                                 LIB_LFP              0  
                                                                 LIB_LTO              0  
...                                                                                 ...  
                         BEV        2050   ssp585 V1B            LSB                  0  
                                                  F2_fuel_diesel diesel_mix           0  
                                                  F2_fuel_elec   REC_100PV            1  
                                                                 grid_elec            1  
                                                  V2_oper        dummy_tech           0  

[240 rows x 3 columns]

#### save mapped TRL, which will be called during LCA modeling 

In [7]:
ddxrr = data2.to_xarray() 
ddxrr

<xarray.Dataset>
Dimensions:     (v_type: 1, size: 1, powertrain: 2, v_year: 3, SSP: 4,
                 Module: 5, Tech: 9)
Coordinates:
  * v_type      (v_type) object 'garbage_truck'
  * size        (size) object 'HDV_class7'
  * powertrain  (powertrain) object 'BEV' 'ICEV-d'
  * v_year      (v_year) int64 2030 2040 2050
  * SSP         (SSP) object 'ssp119' 'ssp126' 'ssp245' 'ssp585'
  * Module      (Module) object 'F2_fuel_diesel' 'F2_fuel_elec' ... 'V2_oper'
  * Tech        (Tech) object 'LIB_LFP' 'LIB_LTO' ... 'dummy_tech' 'grid_elec'
Data variables:
    TRL         (v_type, size, powertrain, v_year, SSP, Module, Tech) float64 ...
    tech_appl   (v_type, size, powertrain, v_year, SSP, Module, Tech) float64 ...
    tech_avail  (v_type, size, powertrain, v_year, SSP, Module, Tech) float64 ...

In [8]:
ddxrr.to_netcdf('../data/to_use_interm/TRL_1C_vehDSwhTRL.nc')

#### see the final mapped data and xarray
- when selecting a powertrain, AND filter through tech_appl score == 1, you'll see mappped powertrain and tech

In [9]:
vdd36 = ddxrr.sel(v_type = "garbage_truck", powertrain = "BEV")
vdd36.where(vdd36['tech_appl'] == 1, drop=True)

<xarray.Dataset>
Dimensions:     (size: 1, v_year: 3, SSP: 4, Module: 3, Tech: 8)
Coordinates:
    v_type      <U13 'garbage_truck'
  * size        (size) object 'HDV_class7'
    powertrain  <U3 'BEV'
  * v_year      (v_year) int64 2030 2040 2050
  * SSP         (SSP) object 'ssp119' 'ssp126' 'ssp245' 'ssp585'
  * Module      (Module) object 'F2_fuel_elec' 'V1A' 'V1B'
  * Tech        (Tech) object 'LIB_LFP' 'LIB_LTO' ... 'dummy_tech' 'grid_elec'
Data variables:
    TRL         (size, v_year, SSP, Module, Tech) float64 nan nan ... nan nan
    tech_appl   (size, v_year, SSP, Module, Tech) float64 nan nan ... nan nan
    tech_avail  (size, v_year, SSP, Module, Tech) float64 nan nan ... nan nan

In [10]:
vdd336 = ddxrr.sel(v_type = "garbage_truck", powertrain = "ICEV-d")
vdd336.where(vdd336['tech_appl'] == 1, drop=True)

<xarray.Dataset>
Dimensions:     (size: 1, v_year: 3, SSP: 4, Module: 3, Tech: 2)
Coordinates:
    v_type      <U13 'garbage_truck'
  * size        (size) object 'HDV_class7'
    powertrain  <U6 'ICEV-d'
  * v_year      (v_year) int64 2030 2040 2050
  * SSP         (SSP) object 'ssp119' 'ssp126' 'ssp245' 'ssp585'
  * Module      (Module) object 'F2_fuel_diesel' 'V1A' 'V2_oper'
  * Tech        (Tech) object 'diesel_mix' 'dummy_tech'
Data variables:
    TRL         (size, v_year, SSP, Module, Tech) float64 9.0 nan ... nan 9.0
    tech_appl   (size, v_year, SSP, Module, Tech) float64 1.0 nan ... nan 1.0
    tech_avail  (size, v_year, SSP, Module, Tech) float64 1.0 nan ... nan 1.0

#### see whatever available module / tech combination for a specified vehicle

In [11]:
cs = prepare_veh_tech( v_data =  data2 )

In [12]:
# returned: vd_dict_xr, vd_dict_list, vd_dict_list2_right_dimupto4th
vd3 , vd6,  vd63 = cs.get_V_P_avail_comb_dict()

available tech. for vehicle garbage_truck and powertrain BEV for all years and SSPx before_mod/Tech combination: 96
available tech. for vehicle garbage_truck and powertrain ICEV-d for all years and SSPx before_mod/Tech combination: 24


In [13]:
vd3["garbage_truck"]["BEV"]

<xarray.Dataset>
Dimensions:     (size: 1, v_year: 3, SSP: 4, Module: 3, Tech: 8)
Coordinates:
    v_type      <U13 'garbage_truck'
  * size        (size) object 'HDV_class7'
    powertrain  <U3 'BEV'
  * v_year      (v_year) int64 2030 2040 2050
  * SSP         (SSP) object 'ssp119' 'ssp126' 'ssp245' 'ssp585'
  * Module      (Module) object 'F2_fuel_elec' 'V1A' 'V1B'
  * Tech        (Tech) object 'LIB_LFP' 'LIB_LTO' ... 'dummy_tech' 'grid_elec'
Data variables:
    TRL         (size, v_year, SSP, Module, Tech) float64 nan nan ... nan nan
    tech_appl   (size, v_year, SSP, Module, Tech) float64 nan nan ... nan nan
    tech_avail  (size, v_year, SSP, Module, Tech) float64 nan nan ... nan nan

### visualize final TRL table

In [14]:
xx = cs.get_tech_TRL_dict(to_display = True)

for the technology LIB_LFP the TRL table is: 


,ssp119,ssp126,ssp245,ssp585
2030,9,9,9,9
2040,9,9,9,9
2050,9,9,9,9


for the technology LIB_LTO the TRL table is: 


,ssp119,ssp126,ssp245,ssp585
2030,9,9,9,9
2040,9,9,9,9
2050,9,9,9,9


for the technology LIB_NCA the TRL table is: 


...

,ssp119,ssp126,ssp245,ssp585
2030,9,9,9,9
2040,9,9,9,9
2050,9,9,9,9


for the technology dummy_tech the TRL table is: 


,ssp119,ssp126,ssp245,ssp585
2030,9,9,9,9
2040,9,9,9,9
2050,9,9,9,9


for the technology grid_elec the TRL table is: 


,ssp119,ssp126,ssp245,ssp585
2030,9,9,9,9
2040,9,9,9,9
2050,9,9,9,9
